In [1]:
import torch
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
import pickle

In [2]:
class Loader:
    
    def unpickle(self,file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    
    def load_train_data(self):
        '''
        loads training data: 50,000 examples with 3072 features
        '''
        X_train = None
        Y_train = None
        for i in range(1, 6):
            pickleFile = self.unpickle('cifar-10-batches-py/data_batch_{}'.format(i))
            dataX = pickleFile[b'data']
            dataY = pickleFile[b'labels']
            if type(X_train) is np.ndarray:
                X_train = np.concatenate((X_train, dataX))
                Y_train = np.concatenate((Y_train, dataY))
            else:
                X_train = dataX
                Y_train = dataY

        Y_train = Y_train.reshape(Y_train.shape[0], 1)

        return X_train, Y_train

    def load_test_data(self):
        '''
        loads testing data: 10,000 examples with 3072 features
        '''
        X_test = None
        Y_test = None
        pickleFile = self.unpickle('cifar-10-batches-py/test_batch')
        dataX = pickleFile[b'data']
        dataY = pickleFile[b'labels']
        if type(X_test) is np.ndarray:
            X_test = np.concatenate((X_test, dataX))
            Y_test = np.concatenate((Y_test, dataY))
        else:
            X_test = np.array(dataX)
            Y_test = np.array(dataY)

        Y_test = Y_test.reshape(Y_test.shape[0], 1)

        return X_test, Y_test

In [3]:
# split data
X_trainval,Y_trainval = Loader().load_train_data()
X_test, Y_test = Loader().load_test_data()

X_trainval = X_trainval.astype(float)
X_trainval_scale = X_trainval / 255

X_test = X_test.astype(float)
X_test_scale = X_test / 255

X_trainval_reshape = X_trainval_scale.reshape((X_trainval.shape[0],3,32,32))
X_test_reshape = X_test_scale.reshape((X_test.shape[0],3,32,32))


num_train = 50000
indices = list(range(num_train))
split = 40000
np.random.shuffle(indices)

valid_idx, train_idx = indices[split:], indices[:split]

X_train_reshape = X_trainval_reshape[train_idx]
Y_train_reshape = Y_trainval[train_idx]

X_val_reshape = X_trainval_reshape[valid_idx]
Y_val_reshape = Y_trainval[valid_idx]



In [ ]:
Y_train_reshape.reshape(40000).shape

In [ ]:
print(X_train_reshape.shape)
print(Y_train_reshape.shape)

In [4]:
import torch.utils.data as data_utils

train_try = data_utils.TensorDataset(torch.FloatTensor(X_train_reshape), torch.LongTensor(Y_train_reshape.reshape(40000)))
train_try_loader = data_utils.DataLoader(train_try, batch_size=4, shuffle=True)

In [ ]:
for i, data in enumerate(train_try_loader, 0):
    # get the inputs
    inputs, labels = data
    print(inputs)
    print(labels)
    break

In [121]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 3,padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 32, 3,padding=1)
        
        self.conv3 = nn.Conv2d(32,64,3,padding=1)


        self.fc1 = nn.Linear(64*4*4,120)
        self.fc2 = nn.Linear(120,96)
        self.fc3 = nn.Linear(96,10)
        
        self.bn1 = nn.BatchNorm2d(10)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        
        self.drop = nn.Dropout2d(p=0.7)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        #x = self.pool(F.relu(self.bn1(self.conv1(x))))
        #x = self.pool(F.relu(self.bn2(self.conv2(x))))
        
        
        x = self.pool(F.relu(self.conv2(x)))
        
        x = self.pool(F.relu(self.conv3(x)))
        #x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        #x = self.pool(F.relu(self.conv4(x)))
        #print(x)

        
        x = x.view(-1,64*4*4)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        #x = self.softmax(x)
        return x
    
net = Net()

import torch.optim as optim
from random import randint
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [122]:
temp_lr = 0.001
prev_acc = 0


best_net = Net()
train_loss = []
validation_acc = []
best_acc = 0
for epoch in range(15):  # loop over the dataset multiple times
    if epoch > 0:
        #if (prev_acc / acc > 0.995):
        #    temp_lr /= 5
        #prev_acc = acc
        if (epoch % 10 == 0):
            temp_lr /= 4
    optimizer = optim.SGD(net.parameters(), lr=temp_lr, momentum=0.9)
    #temp_lr /= 5
    
    running_loss = 0.0
    
    train_correct = 0
    train_total = 0
    for batch_idx, (inputs, targets) in enumerate(train_try_loader):
        
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        
        _, predicted_train = torch.max(outputs.data, 1)
        
        for i in range(4):
            if predicted_train[i] == int(targets[i].data.numpy()[0]):
                train_correct += 1
            train_total +=1

        # print statistics
        running_loss += loss.data[0]
        #print("Summary: %d is %.3f" % (epoch, acc))
        
    train_acc = train_correct / train_total
        
        
    correct = 0
    total = 0
    for i in range(0,10000):
        inputs = torch.FloatTensor(X_val_reshape[i].reshape((1,3,32,32)))
        labels = torch.LongTensor(Y_val_reshape[i])
        outputs = net(Variable(inputs))
        _, predicted = torch.max(outputs.data, 1)

        if predicted[0] == labels[0]:
            correct += 1
        total += 1
        
    acc = correct / total
    train_loss.append(running_loss/10000)
    validation_acc.append(acc)
    if (acc > best_acc):
        best_acc = acc
        best_net.load_state_dict(net.state_dict())
    
    print("epoch: %d loss: %.3f train_acc: %.3f val_acc: %.3f learning rate: %.6f" % (epoch,running_loss/10000,train_acc,acc,temp_lr))

print('Finished Training')

epoch: 0 loss: 2.090 train_acc: 0.215 val_acc: 0.360 learning rate: 0.001000
epoch: 1 loss: 1.518 train_acc: 0.449 val_acc: 0.500 learning rate: 0.001000
epoch: 2 loss: 1.244 train_acc: 0.557 val_acc: 0.573 learning rate: 0.001000
epoch: 3 loss: 1.071 train_acc: 0.617 val_acc: 0.627 learning rate: 0.001000
epoch: 4 loss: 0.961 train_acc: 0.660 val_acc: 0.623 learning rate: 0.001000
epoch: 5 loss: 0.868 train_acc: 0.695 val_acc: 0.637 learning rate: 0.001000
epoch: 6 loss: 0.794 train_acc: 0.718 val_acc: 0.651 learning rate: 0.001000
epoch: 7 loss: 0.723 train_acc: 0.744 val_acc: 0.672 learning rate: 0.001000
epoch: 8 loss: 0.662 train_acc: 0.764 val_acc: 0.664 learning rate: 0.001000
epoch: 9 loss: 0.606 train_acc: 0.785 val_acc: 0.659 learning rate: 0.001000
epoch: 10 loss: 0.366 train_acc: 0.872 val_acc: 0.709 learning rate: 0.000250
epoch: 11 loss: 0.294 train_acc: 0.897 val_acc: 0.705 learning rate: 0.000250
epoch: 12 loss: 0.246 train_acc: 0.915 val_acc: 0.706 learning rate: 0.000

In [123]:
correct = 0
total = 0
for i in range(0,10000):
    inputs = torch.FloatTensor(X_test_reshape[i].reshape((1,3,32,32)))
    labels = torch.LongTensor(Y_test[i])
    outputs = best_net(Variable(inputs))
    _, predicted = torch.max(outputs.data, 1)
    
    if predicted[0] == labels[0]:
        correct += 1
    total += 1

print(correct)
print(total)
print(correct/total)

7062
10000
0.7062


In [131]:
train_loss

[2.0899593315303324,
 1.5176347495943308,
 1.2436639948002994,
 1.0712805995244532,
 0.9608085837502033,
 0.8682140761709772,
 0.7941201634490047,
 0.7226411649866729,
 0.6621319993035868,
 0.6055460957959993,
 0.36587994550366676,
 0.2937783408456708,
 0.24561901038416867,
 0.20516464548483682,
 0.16961843757615463]

In [132]:
validation_acc

[0.3603,
 0.5003,
 0.5733,
 0.627,
 0.6227,
 0.6368,
 0.6512,
 0.672,
 0.6645,
 0.6589,
 0.7091,
 0.705,
 0.7055,
 0.7016,
 0.7022]